# Load the preliminary code

In [2]:
import matplotlib.pyplot as plt
from sympy import Poly
import sympy as sym
import numpy as np
from numpy import c_

# We'll specify that x and t are variables:
x, t = sym.symbols('x, t')

ll = "loaded"

In [3]:
lll = " correctly!"

def rref(matrix):
    A = np.array(matrix, dtype=np.float64)
    
    r = 0 #row
    c = 0 #column

    (maxr, maxc) = A.shape

    prev_row_pivot = -1   # The previous row with a leading term:
    tmp_pivot_row = -1
    this_pivot_row = -1

    for c in range(maxc):  # go through each column
      tmp_pivot_row = -1  # reset the tmp pivot row
      this_pivot_row = -1  # reset this pivot row
      for r in range(maxr):  # Find the leading term in this column below prev_row_pivot
        if ((r > prev_row_pivot) and (A[r][c] != 0.0)):
          tmp_pivot_row = r  # find the new pivot row
      if (tmp_pivot_row > -1):  # we found a candidate for the leading term:
        #swap row this_pivot_row with (prev_row_pivot + 1)
        A[[tmp_pivot_row, prev_row_pivot+1]] = A[[prev_row_pivot + 1, tmp_pivot_row]]
        #set this_pivot_row to prev_row_pivot+1
        this_pivot_row = prev_row_pivot + 1
        #Now, normalize the row based on the pivot value:
        A[ this_pivot_row ] = A[ this_pivot_row ] / A[ this_pivot_row ][c]
        #Then, use this row to force 0's above and below the pivot entry:
        #subtract multiples of of the this_pivot_row row from the others
        for r_ in range(len(A)):
            if r_ != this_pivot_row:
                A[r_] = A[r_] - A[this_pivot_row] * A[r_][c] / A[this_pivot_row][c]
        prev_row_pivot = this_pivot_row
    return A    
    
    

print(">> The page ", ll, lll)

>> The page  loaded  correctly!


# Set up the secret number

You'll encode a secret number $s$ into a 4th or 5th degree polynomial as its vertical intercept.

Once you've chosen $s$, then create a 4th or 5th degree polynomial $f$ such that the vertical intercept is $s$.


In [4]:
# the secret number is s:

s = 14

p = Poly(14+2*x + 3*x**18, x, domain='RR')

p

Poly(3.0*x**18 + 2.0*x + 14.0, x, domain='RR')

In [5]:
#  Change this to evaluate the polynomial at 0:
p.eval(x, 0)

# Check that this is your secret number:

14.0000000000000

# Generate your data points

Use this code to generate the data points for your polynomial.  Write the data on the second page of your handout.  

Writing 

```
p.eval(x, 3)
```

will evaluate $p$ when $x = 3$.  Just change the $3$ to any value to evaluate $p$ at that input.

Remember:  Keep your secret number and polynomial hidden.

In [20]:
p.eval(x, 3)

1162261487.00000

#  Decode the polynomial from another group

In this section, we set up a linear system to find the polynomial and the secret number which was given to us by another group.

First, set up a linear algebra system to find the polynomial on the handout and then enter the matrix here.  You may have to adjust the size of the matrix below to accommodate your data.

In [6]:
# create the matrix:
A = np.matrix([[(-1)**5, (-1)**4, (-1)**3, (-1)**2, -1, 1],
               [ (1)**5,  (1)**4,  (1)**3,  (1)**2,  1, 1],
               [ (2)**5,  (2)**4,  (2)**3,  (2)**2,  2, 1],
               [ (4)**5,  (4)**4,  (4)**3,  (4)**2,  4, 1],
               [ (3)**5,  (3)**4,  (3)**3,  (3)**2,  3, 1],
               [ (7)**5,  (7)**4,  (7)**3,  (7)**2,  7, 1]])
 
# create the vector for the right hand side:
b = np.matrix([[8], [14], [58], [272], [148], [368]])

#Make sure the system looks appropriate:

print(np.c_[A,b])

[[   -1     1    -1     1    -1     1     8]
 [    1     1     1     1     1     1    14]
 [   32    16     8     4     2     1    58]
 [ 1024   256    64    16     4     1   272]
 [  243    81    27     9     3     1   148]
 [16807  2401   343    49     7     1   368]]


##  Solve using Row Operations:

We'll solve the augmented matrix  $\left( A \, | \, \vec{b} \right)$ using the function

```
rref(np.c_[A,b])
```

to provide the reduced row echelon form of the matrix system $\left(A \, | \, \vec{b} \right)$.


In [7]:
print(rref(np.c_[A,b]))

[[ 1.          0.          0.          0.          0.          0.
  -0.01666667]
 [ 0.          1.          0.          0.          0.          0.
  -0.71666667]
 [ 0.          0.          1.          0.          0.          0.
   6.25      ]
 [-0.         -0.         -0.          1.          0.          0.
   4.91666667]
 [ 0.          0.          0.          0.          1.          0.
  -3.23333333]
 [ 0.          0.          0.          0.          0.          1.
   6.8       ]]


## Solve the system using numerical methods:

We can also generate a solution using some more advanced numerical linear algebra techniques contained inside Numpy.

To solve it this way, use:



```
np.linalg.solve(A,b)
```

or

```
np.linalg.lstsq(A,b,rcond=None)
```

The first method requires the matrix $A$ to be square.  The second method is a bit more robust, and can work with any size matrix.


In [8]:
# Solve the system using a numerical linear algebra solution:
solve_sol = np.linalg.solve(A,b)

print("The solution using np.linalg.solve(A,b) is ")
print(solve_sol)

The solution using np.linalg.solve(A,b) is 
[[-0.01666667]
 [-0.71666667]
 [ 6.25      ]
 [ 4.91666667]
 [-3.23333333]
 [ 6.8       ]]


In [9]:
# Solve the system using the numerical least squares method:
lstsq_sol = np.linalg.lstsq(A,b,rcond=None)

print("The np.linalg.lstsq(A,b) can work with non-square matrices A")
print("It gives back multiple pieces of information:")
print("lstsq_sol = ")
print(lstsq_sol)

print("")
print("The solution is first piece returned.  lstsq_sol[0] = ")
print(lstsq_sol[0])


The np.linalg.lstsq(A,b) can work with non-square matrices A
It gives back multiple pieces of information:
lstsq_sol = 
(matrix([[-0.01666667],
        [-0.71666667],
        [ 6.25      ],
        [ 4.91666667],
        [-3.23333333],
        [ 6.8       ]]), matrix([], shape=(1, 0), dtype=float64), 6, array([1.70155760e+04, 1.28731256e+02, 5.91300489e+00, 2.13918002e+00,
       9.53598477e-01, 1.56965136e-01]))

The solution is first piece returned.  lstsq_sol[0] = 
[[-0.01666667]
 [-0.71666667]
 [ 6.25      ]
 [ 4.91666667]
 [-3.23333333]
 [ 6.8       ]]
